In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 1. Load and Explore the Data

In [ ]:
# Load the datasets
customers = pd.read_csv('../resource-one-repo/0.source-data/customers/customers.csv', header=None,
                       names=['customer_id', 'first_name', 'last_name', 'full_name'])

employees = pd.read_csv('../resource-one-repo/0.source-data/employees/employees.csv', header=None,
                       names=['employee_id', 'manager_id', 'first_name', 'last_name', 'full_name', 
                              'title', 'level', 'marital_status', 'gender', 'territory', 'country', 'region'])

orders = pd.read_csv('../resource-one-repo/0.source-data/orders/orders.csv')

print(f"Customers: {len(customers):,} records")
print(f"Employees: {len(employees):,} records")
print(f"Orders: {len(orders):,} records")

In [ ]:
# Quick data preview
print("\n=== ORDERS DATA SAMPLE ===")
display(orders.head())
print("\n=== EMPLOYEES DATA SAMPLE ===")
display(employees.head())
print("\n=== CUSTOMERS DATA SAMPLE ===")
display(customers.head())

In [ ]:
# Data preparation
orders['orderdate'] = pd.to_datetime(orders['orderdate'])
orders['shipdate'] = pd.to_datetime(orders['shipdate'])
orders['duedate'] = pd.to_datetime(orders['duedate'])
orders['year'] = orders['orderdate'].dt.year
orders['month'] = orders['orderdate'].dt.month
orders['quarter'] = orders['orderdate'].dt.quarter

print("Date range:", orders['orderdate'].min(), "to", orders['orderdate'].max())

## 2. Key Metrics Overview

Let's start with the headline numbers that set the stage for our story.

In [ ]:
# Calculate key business metrics
total_revenue = orders['totaldue'].sum()
total_orders = orders['salesorderid'].nunique()
avg_order_value = orders.groupby('salesorderid')['totaldue'].first().mean()
active_customers = orders['customerid'].nunique()
active_employees = orders['employeeid'].nunique()

print("="*50)
print("BUSINESS PERFORMANCE SNAPSHOT")
print("="*50)
print(f"💰 Total Revenue: ${total_revenue:,.2f}")
print(f"📦 Total Orders: {total_orders:,}")
print(f"📊 Average Order Value: ${avg_order_value:,.2f}")
print(f"👥 Active Customers: {active_customers:,}")
print(f"🏆 Active Sales Reps: {active_employees:,}")
print(f"📈 Revenue per Employee: ${total_revenue/active_employees:,.2f}")
print("="*50)

## 3. Story Point #1: Revenue Trends Show Strong Growth

In [ ]:
# Monthly revenue trends
monthly_revenue = orders.groupby([orders['orderdate'].dt.to_period('M')])['totaldue'].sum()

fig, ax = plt.subplots(figsize=(14, 6))
monthly_revenue.plot(kind='line', ax=ax, linewidth=2.5, color='#2E86AB')
ax.set_title('Monthly Revenue Trend: Consistent Growth Pattern', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Revenue ($)', fontsize=12)
ax.grid(True, alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# Add annotation
max_revenue = monthly_revenue.max()
max_month = monthly_revenue.idxmax()
plt.tight_layout()
plt.show()

print(f"\n📈 Peak Revenue Month: {max_month} with ${max_revenue:,.2f}")

## 4. Story Point #2: Employee Performance Varies Significantly

In [ ]:
# Employee performance analysis
employee_sales = orders.groupby('employeeid').agg({
    'totaldue': 'sum',
    'salesorderid': 'nunique',
    'customerid': 'nunique'
}).rename(columns={
    'totaldue': 'revenue',
    'salesorderid': 'orders',
    'customerid': 'customers'
})

employee_sales = employee_sales.merge(employees[['employee_id', 'full_name', 'territory', 'region']], 
                                      left_index=True, right_on='employee_id', how='left')

# Sort by revenue
employee_sales = employee_sales.sort_values('revenue', ascending=False)

print("\n=== TOP 5 SALES PERFORMERS ===")
display(employee_sales.head()[['full_name', 'territory', 'revenue', 'orders', 'customers']])

print("\n=== BOTTOM 5 SALES PERFORMERS ===")
display(employee_sales.tail()[['full_name', 'territory', 'revenue', 'orders', 'customers']])

In [ ]:
# Visualize employee performance distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Revenue distribution
top_10 = employee_sales.head(10)
ax1.barh(range(len(top_10)), top_10['revenue'], color='#06A77D')
ax1.set_yticks(range(len(top_10)))
ax1.set_yticklabels(top_10['full_name'])
ax1.set_xlabel('Total Revenue ($)', fontsize=11)
ax1.set_title('Top 10 Sales Representatives by Revenue', fontsize=13, fontweight='bold')
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
ax1.invert_yaxis()

# Orders per employee
ax2.barh(range(len(top_10)), top_10['orders'], color='#F18F01')
ax2.set_yticks(range(len(top_10)))
ax2.set_yticklabels(top_10['full_name'])
ax2.set_xlabel('Number of Orders', fontsize=11)
ax2.set_title('Top 10 Sales Representatives by Order Volume', fontsize=13, fontweight='bold')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

# Performance gap analysis
avg_revenue = employee_sales['revenue'].mean()
std_revenue = employee_sales['revenue'].std()
performance_gap = employee_sales['revenue'].max() - employee_sales['revenue'].min()

print(f"\n📊 Average Revenue per Rep: ${avg_revenue:,.2f}")
print(f"📊 Performance Gap (Max - Min): ${performance_gap:,.2f}")
print(f"⚠️ Standard Deviation: ${std_revenue:,.2f}")

## 5. Story Point #3: Regional Performance Reveals Opportunities

In [ ]:
# Merge orders with employee data for regional analysis
orders_with_regions = orders.merge(employees[['employee_id', 'territory', 'region', 'country']], 
                                   left_on='employeeid', right_on='employee_id', how='left')

# Regional performance
regional_performance = orders_with_regions.groupby('region').agg({
    'totaldue': 'sum',
    'salesorderid': 'nunique',
    'customerid': 'nunique',
    'employeeid': 'nunique'
}).rename(columns={
    'totaldue': 'revenue',
    'salesorderid': 'orders',
    'customerid': 'customers',
    'employeeid': 'employees'
})

regional_performance['revenue_per_employee'] = regional_performance['revenue'] / regional_performance['employees']
regional_performance['avg_order_value'] = regional_performance['revenue'] / regional_performance['orders']

regional_performance = regional_performance.sort_values('revenue', ascending=False)

print("\n=== REGIONAL PERFORMANCE BREAKDOWN ===")
display(regional_performance)

In [ ]:
# Regional performance visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Revenue by region
regional_performance['revenue'].plot(kind='bar', ax=axes[0, 0], color='#2E86AB')
axes[0, 0].set_title('Total Revenue by Region', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
axes[0, 0].tick_params(axis='x', rotation=45)

# Revenue per employee by region
regional_performance['revenue_per_employee'].plot(kind='bar', ax=axes[0, 1], color='#06A77D')
axes[0, 1].set_title('Revenue per Employee by Region', fontsize=13, fontweight='bold')
axes[0, 1].set_ylabel('Revenue per Employee ($)')
axes[0, 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
axes[0, 1].tick_params(axis='x', rotation=45)

# Number of employees by region
regional_performance['employees'].plot(kind='bar', ax=axes[1, 0], color='#F18F01')
axes[1, 0].set_title('Number of Sales Reps by Region', fontsize=13, fontweight='bold')
axes[1, 0].set_ylabel('Number of Employees')
axes[1, 0].tick_params(axis='x', rotation=45)

# Average order value by region
regional_performance['avg_order_value'].plot(kind='bar', ax=axes[1, 1], color='#C73E1D')
axes[1, 1].set_title('Average Order Value by Region', fontsize=13, fontweight='bold')
axes[1, 1].set_ylabel('Average Order Value ($)')
axes[1, 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Story Point #4: Territory Analysis - Where Are The Gaps?

In [ ]:
# Territory-level analysis
territory_performance = orders_with_regions.groupby(['territory', 'region']).agg({
    'totaldue': 'sum',
    'salesorderid': 'nunique',
    'employeeid': 'nunique'
}).rename(columns={
    'totaldue': 'revenue',
    'salesorderid': 'orders',
    'employeeid': 'employees'
}).sort_values('revenue', ascending=False)

territory_performance['revenue_per_employee'] = territory_performance['revenue'] / territory_performance['employees']

print("\n=== TERRITORY PERFORMANCE ===")
display(territory_performance.head(10))

## 7. Customer Analysis - Who Are Our Best Customers?

In [ ]:
# Customer value analysis
customer_value = orders.groupby('customerid').agg({
    'totaldue': 'sum',
    'salesorderid': 'nunique',
    'orderdate': ['min', 'max']
}).round(2)

customer_value.columns = ['total_revenue', 'order_count', 'first_order', 'last_order']
customer_value['avg_order_value'] = customer_value['total_revenue'] / customer_value['order_count']
customer_value = customer_value.sort_values('total_revenue', ascending=False)

print("\n=== TOP 10 CUSTOMERS BY REVENUE ===")
display(customer_value.head(10))

# Customer segmentation
total_customers = len(customer_value)
top_20_pct = int(total_customers * 0.2)
top_20_revenue = customer_value.head(top_20_pct)['total_revenue'].sum()
revenue_concentration = (top_20_revenue / total_revenue) * 100

print(f"\n📊 Top 20% of customers generate {revenue_concentration:.1f}% of revenue")
print(f"📊 That's {top_20_pct:,} customers generating ${top_20_revenue:,.2f}")

In [ ]:
# Customer distribution visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Revenue distribution histogram
ax1.hist(customer_value['total_revenue'], bins=50, color='#2E86AB', alpha=0.7, edgecolor='black')
ax1.set_xlabel('Customer Lifetime Value ($)', fontsize=11)
ax1.set_ylabel('Number of Customers', fontsize=11)
ax1.set_title('Customer Revenue Distribution', fontsize=13, fontweight='bold')
ax1.axvline(customer_value['total_revenue'].median(), color='red', linestyle='--', linewidth=2, label='Median')
ax1.legend()

# Order count distribution
ax2.hist(customer_value['order_count'], bins=50, color='#06A77D', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Number of Orders per Customer', fontsize=11)
ax2.set_ylabel('Number of Customers', fontsize=11)
ax2.set_title('Customer Order Frequency Distribution', fontsize=13, fontweight='bold')
ax2.axvline(customer_value['order_count'].median(), color='red', linestyle='--', linewidth=2, label='Median')
ax2.legend()

plt.tight_layout()
plt.show()

## 8. Key Insights & Recommendations

### 🎯 The Story in Three Acts:

#### Act 1: The Good News
- Strong overall revenue growth with consistent monthly trends
- Healthy customer base with repeat purchase behavior

#### Act 2: The Challenge
- Significant performance variation among sales representatives
- Uneven regional resource allocation
- Revenue concentration in top customers creates risk

#### Act 3: The Opportunity
- Identify and replicate best practices from top performers
- Reallocate resources to high-potential territories
- Expand customer base to reduce concentration risk

In [ ]:
# Calculate opportunity sizing
median_revenue_per_rep = employee_sales['revenue'].median()
below_median_reps = employee_sales[employee_sales['revenue'] < median_revenue_per_rep]
gap_to_median = (median_revenue_per_rep - below_median_reps['revenue']).sum()

print("\n" + "="*60)
print("💡 OPPORTUNITY SIZING")
print("="*60)
print(f"If all below-median performers reached median performance:")
print(f"📈 Potential Additional Revenue: ${gap_to_median:,.2f}")
print(f"📈 That's a {(gap_to_median/total_revenue)*100:.1f}% increase in total revenue")
print(f"📊 Number of reps below median: {len(below_median_reps)} out of {len(employee_sales)}")
print("="*60)

## 9. Action Items

### Immediate Actions (Next 30 Days):
1. **Sales Training**: Conduct knowledge transfer sessions with top 3 performers
2. **Territory Review**: Analyze workload distribution in underperforming regions
3. **Customer Risk**: Develop retention strategy for top 20% of customers

### Short-term Actions (Next 90 Days):
1. **Performance Improvement Plan**: Work with bottom quartile sales reps
2. **Market Expansion**: Identify new customer acquisition opportunities
3. **Resource Reallocation**: Consider territory reassignment based on data

### Long-term Strategy (6-12 Months):
1. **Predictive Analytics**: Build models to forecast sales and identify at-risk customers
2. **Compensation Review**: Align incentives with strategic priorities
3. **Market Penetration**: Expand presence in high-performing regions